<a href="https://colab.research.google.com/github/remixwithkj/Backupmac/blob/main/Assignment_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,161 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jam

In [4]:
import sys # To acess system specific information about undelying interpreter and OS.
from pyspark import SparkConf, SparkContext # Importing classes SparkConf, and SparkContext to work with Spark in Python.
from math import sqrt # Importing sqrt function from math module in core Python

In [5]:
def loadMovieNames(): # Defines a function named loadMovieNames
    movieNames = {} # Initialising an empty dictionary named movieNames
    with open("/home/cloudera/moviedata/itemfile.txt") as f: #reads the file at the path defined and store it in file object f
        for line in f: #loop to iterate through each line
            fields = line.split('|') #splits lines into fields wherever encounters pipe i.e. |
            movieNames[int(fields[0])] = fields[1].decode('ascii', 'ignore') #appends movie id as key and movie name as value in the movieNames dictionary initialised earlier
    return movieNames # returns the dictionary with key-value pair of movie id and name respectively


In [ ]:
def makePairs( (user, ratings) ): # Defining fuction makePairs to accept a tuple argument where user is a user_id while ratings is a list of tuples each tuple has movie Id and rating by the user
  (movie1, rating1) = ratings[0] # unpacks first item (0 index) in rating and assigns values to movie1 and rating1 respectively
  (movie2, rating2) = ratings[1] # unpacks second item (1st index) in rating and assigns values to movie1 and rating1 respectively
  return ((movie1, movie2), (rating1, rating2)) # returns a nested tuple, with movie pair and their correpinding ratings

In [ ]:
def filterDuplicates( (userID, ratings) ): # Defining fuction filterDuplicates to accept a tuple argument where user is a user_id while ratings is a list of tuples each tuple has movie Id and rating by the user
    (movie1, rating1) = ratings[0] # unpacks first item (0 index) in rating and assigns values to movie1 and rating1 respectively
    (movie2, rating2) = ratings[1] # unpacks second item (1st index) in rating and assigns values to movie1 and rating1 respectively
    return movie1 < movie2 # Compares movie IDs and returns True if movie ID 2 is greater than movie ID1. This is to reject duplicate movie pairs

In [8]:
# CosineSimilarity is a measure to calculate distance rather simnilarity between two
# sparse vectors, it is widely used by recommendation systems.
# The formula for Cosine similarity is Cosinesimilarity = (v1 . V2)/(|V1||V2|)
# That is Dot product of vectors divided by mod/magnitude of the vectors.

def computeCosineSimilarity(ratingPairs): # Defining a function to accept rating pairs tuple as input
    numPairs = 0 #initialising counter value to 0
    sum_xx = sum_yy = sum_xy = 0 # initialising the values to 0 for future result storage in the 'for' loop
    for ratingX, ratingY in ratingPairs: # 'for' loop to extract ratingX and ratingY from tuple ratingPairs one by one
        sum_xx += ratingX * ratingX # multiplies ratingX by itself and accumulates the sum in sum_xx variable
        sum_yy += ratingY * ratingY # multiplies ratingY by itself and accumulates the sum in sum_yy variable
        sum_xy += ratingX * ratingY # multiplies ratingY by ratingX and accumulates the sum in sum_xy variable
        numPairs += 1 # counter for the number of time the loop runs

    numerator = sum_xy # sum_xy is assigned to numerator
    denominator = sqrt(sum_xx) * sqrt(sum_yy) # square roots of sum_xx and sum_yy is assigned to denominator

    score = 0 # score is initialised with the value of 0
    if (denominator): # if denominator is not 0 that is if denomonator is True then score is calcuated in the next line
        score = (numerator / (float(denominator))) # score calculation, equivalent of Cosine similarity

    return (score, numPairs) # the final value of score and the counter value is returned by the function

In [ ]:
conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities") # creates a new Spark object and names the App as MovieSimilarities
sc = SparkContext(conf = conf) #entry point for working with spark for connecting with a spark cluster

In [ ]:
# print "\nLoading movie names..."
nameDict = loadMovieNames() # calls loadMovieNames function defined earlier to return with key-value pair of movie id and name respectively
data = sc.textFile("file:///home/cloudera/moviedata/datafile2.txt") # load movie data from datafile2.txt into the RDD called data

In [ ]:
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

# map(lambda l: l.split()) - splits each line of data into list
# map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))) - takes list from first map function and converts
# to tuple (userID, (movieID, rating)) where UseerID is and movieID are integers and rating is float

joinedRatings = ratings.join(ratings)
# ratings RDD self joins into a new RDD giving all pairs of movie rated by the same user in the
# following tuple format (userID, ((movieID1, rating1), (movieID2, rating2))......).

uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)
# run filter function on joinedRatings RDD and retain only the unique movies pairs in uniqueJoinedRatings RDD

moviePairs = uniqueJoinedRatings.map(makePairs)
# calls previously defined function uniqueJoinedRatings and takes a tuple
# in format of (userID, ((movieID1, rating1), (movieID2, rating2))) to returns
# tuple in the format of ((movieID1, movieID2), (rating1, rating2))..

moviePairRatings = moviePairs.groupByKey()
# takes moviepairs and groups them by movie key movieID ((movieID1, movieID2)) in a new RDD

moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()
# takes moviePairRatings RDD and calls computeCosineSimilarity function defined earlier to calculate
# similarity between each movie pair based on their ratings. The cache() function retains
# result in memoory for faster access later.

In [ ]:
if (len(sys.argv) > 1): # checks if there is a command line argument

    scoreThreshold = 0.10 # sets scoreThreshold to 0.1 for similarity
    coOccurenceThreshold = 2 # sets value for coOccurenceThreshold to 2, i.e. how often were 2 movie rated together

    movieID = int(sys.argv[1]) # extracts movieID from command line argument and typecasts it to integer

    filteredResults = moviePairSimilarities.filter(lambda((pair,sim)): \ # filters earlier calculated RDD on the basis of next 2 lines
        (pair[0] == movieID or pair[1] == movieID) \ # checks if either movie in the pair matches specified movieID
        and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold) # and similarity score and coOccurenceThreshold count is above threshold

    results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)
    # map(lambda((pair,sim)): (sim, pair)) - swaps order of sim and pair in the tuple
    # sortByKey(ascending = False) - sorts the result in descending order of similarity score
    # take(10) - Take top 10 results
    # output saved in results

In [ ]:
 print "Top 10 similar movies for " + nameDict[movieID]
 # prints movie using dictionary nameDict[movieID] for which similar movies are being shown
    for result in results: # for loop where 'results' has top 10 similar movies from previous block, result variable iterate through the loop
        (sim, pair) = result # result tuple into similarity score and pair
        similarMovieID = pair[0]
        if (similarMovieID == movieID): #checks if assumed similarMovieID is same as orginal movieID
            similarMovieID = pair[1] # if condition above is true then updates similarMovieID to second movie in the pair
        print nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1])
        # prints name of similar movie from nameDict dictionary using key along with similarity score and co-occurence count